In [1]:
import rosbag
import numpy as np
import matplotlib.pyplot as plt
import glob
import os

/opt/ros/noetic/lib/python3/dist-packages/rosbag/bag.py:1568: SyntaxWarning: invalid escape sequence '\d'
  matches = re.match("#ROS(.*) V(\d).(\d)", version_line)


In [2]:
# Function to process and plot data from a rosbag file
def plot_data_from_bag(bag_file, output_folder, sampling_interval=8):
    # Open the rosbag file
    bag = rosbag.Bag(bag_file)
    
    # Store extracted data
    pedestrian_timestamps = []
    pedestrian_x = []
    pedestrian_y = []
    robot_timestamps = []
    robot_x = []
    robot_y = []
    
    # Extract pedestrian data
    topic_pedestrian = '/social_sim/agent_positions'
    for topic, msg, t in bag.read_messages(topics=[topic_pedestrian]):
        for pose in msg.poses:
            pedestrian_x.append(pose.position.x)
            pedestrian_y.append(pose.position.y)
            pedestrian_timestamps.append(t.to_sec())
    
    pedestrian_normalized_time = (np.array(pedestrian_timestamps) - min(pedestrian_timestamps)) / (max(pedestrian_timestamps) - min(pedestrian_timestamps))
    
    # Extract robot data
    topic_robot = '/tf'
    for topic, msg, t in bag.read_messages(topics=[topic_robot]):
        for transform in msg.transforms:
            if transform.header.frame_id == "map" and transform.child_frame_id == "base_link":
                robot_x.append(transform.transform.translation.x)
                robot_y.append(transform.transform.translation.y)
                robot_timestamps.append(t.to_sec())
    
    robot_normalized_time = (np.array(robot_timestamps) - min(robot_timestamps)) / (max(robot_timestamps) - min(robot_timestamps))
    
    # Undersample the data if necessary
    undersampled_pedestrian_x = pedestrian_x[::sampling_interval]
    undersampled_pedestrian_y = pedestrian_y[::sampling_interval]
    undersampled_pedestrian_normalized_time = pedestrian_normalized_time[::sampling_interval]
    
    undersampled_robot_x = robot_x[::sampling_interval]
    undersampled_robot_y = robot_y[::sampling_interval]
    undersampled_robot_normalized_time = robot_normalized_time[::sampling_interval]
    
    # Create the plot
    plt.figure(figsize=(10, 6))

    # Plot pedestrian positions
    scatter_pedestrian = plt.scatter(
        undersampled_pedestrian_x, 
        undersampled_pedestrian_y, 
        c=undersampled_pedestrian_normalized_time, 
        cmap='viridis', 
        s=50, 
        marker='o', 
        label='Pedestrian Positions'
    )

    # Plot robot positions
    scatter_robot = plt.scatter(
        undersampled_robot_x, 
        undersampled_robot_y, 
        c=undersampled_robot_normalized_time, 
        cmap='viridis', 
        s=50, 
        marker='x', 
        label='Robot Positions'
    )

    # Add colorbars
    plt.colorbar(scatter_pedestrian, label="Timestamp (normalized)")
    #plt.colorbar(scatter_robot, label="Robot Timestamp (normalized)")

    # Labels and title
    plt.xlabel('X Position')
    plt.ylabel('Y Position')
    plt.title(f'Pedestrian and Robot Positions')

    # Ensure equal scaling and add grid
    plt.gca().set_aspect('equal', adjustable='box')
    plt.xlim(-6, 5)  # Adjust as per your data range
    plt.ylim(-16, -5)  # Adjust as per your data range
    plt.grid(True)

    # Show legend
    plt.legend()

    output_filename = os.path.join(output_folder, f"{os.path.basename(bag_file).split('.')[0]}.jpg")
    plt.savefig(output_filename, format='jpg')
    plt.close()  # Close the plot to avoid overlap when saving multiple plots

In [3]:
bag_files = glob.glob('1_person_gaussian_data/data/**/*.bag', recursive=True)

# Print all found bag files
for bag_file in bag_files:
    print(bag_file)

output_folder = "output_plots"

# Loop over each bag file and generate the plot
for bag_file in bag_files:
    plot_data_from_bag(bag_file, output_folder)


1_person_gaussian_data/data/1735837711.5059385-interactive-SmallWarehouse-Random-1.bag
1_person_gaussian_data/data/1735836941.5852273-interactive-SmallWarehouse-Random-1.bag
1_person_gaussian_data/data/1735836658.866576-interactive-SmallWarehouse-Random-1.bag
1_person_gaussian_data/data/1735837244.4592216-interactive-SmallWarehouse-Random-1.bag
1_person_gaussian_data/data/1735838008.1117756-interactive-SmallWarehouse-Random-1.bag
1_person_gaussian_data/data/1735837861.3181496-interactive-SmallWarehouse-Random-1.bag
1_person_gaussian_data/data/1735836727.1371586-interactive-SmallWarehouse-Random-1.bag
1_person_gaussian_data/data/1735837296.9738617-interactive-SmallWarehouse-Random-1.bag
1_person_gaussian_data/data/1735837811.1495419-interactive-SmallWarehouse-Random-1.bag
1_person_gaussian_data/data/1735837500.008841-interactive-SmallWarehouse-Random-1.bag
1_person_gaussian_data/data/1735837960.887146-interactive-SmallWarehouse-Random-1.bag
1_person_gaussian_data/data/1735837447.818205-